In [1]:
# from os import chdir
# from google.colab import drive

# drive.mount("/content/drive", force_remount=True)
# chdir("/content/drive/MyDrive/Eliott/files/")

In [2]:
# IMPORTS

import numpy as np
import pandas as pd




In [3]:
# Ouverture

df_train = pd.read_csv('Corona_NLP_train.csv', sep=',', encoding='latin')

df_test = pd.read_csv('Corona_NLP_test.csv', sep=',', encoding='latin')



In [4]:
df_train.shape

(41157, 6)

In [5]:
df_train.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [6]:
# PREPROCESSING

On vérifie qu'il n'y a pas de null (autre que location) et de ligne dupliquées

In [7]:
df_train.columns

Index(['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet',
       'Sentiment'],
      dtype='object')

In [8]:
df_train = df_train.drop(['Location'], axis=1)
df_test = df_test.drop(['Location'], axis=1)



In [9]:
df_train['Sentiment'].value_counts()

Positive              11422
Negative               9917
Neutral                7713
Extremely Positive     6624
Extremely Negative     5481
Name: Sentiment, dtype: int64

In [10]:
df_train.count()

UserName         41157
ScreenName       41157
TweetAt          41157
OriginalTweet    41157
Sentiment        41157
dtype: int64

In [11]:
train_df2 = df_train.dropna()
test_df2 = df_test.dropna()



In [12]:
train_df2.count()

UserName         41157
ScreenName       41157
TweetAt          41157
OriginalTweet    41157
Sentiment        41157
dtype: int64

In [13]:
train_df3 = train_df2.drop_duplicates()
test_df3 = test_df2.drop_duplicates()

In [14]:
train_df3.count()

UserName         41157
ScreenName       41157
TweetAt          41157
OriginalTweet    41157
Sentiment        41157
dtype: int64

On enlève : 
- Les URLS
- Hashtags
- Mentions
- Mots réservés
- Emojis et smileys


In [15]:
import preprocessor as p


In [16]:
def preprocess_tweet(row):
    text = row['OriginalTweet']
    text = p.clean(text)
    return text

In [17]:
train_df3['OriginalTweet'] = train_df3.apply(preprocess_tweet, axis=1)
test_df3['OriginalTweet'] = test_df3.apply(preprocess_tweet, axis=1)

In [18]:
train_df3.head()


,UserName,ScreenName,TweetAt,OriginalTweet,Sentiment
0,3799,48751,16-03-2020,and and,Neutral
1,3800,48752,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,16-03-2020,"Me, ready to go at supermarket during the outb...",Extremely Negative


In [19]:
train_df3['OriginalTweet'] = train_df3['OriginalTweet'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')
test_df3['OriginalTweet'] = test_df3['OriginalTweet'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')

/tmp/ipykernel_14697/2468492154.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train_df3['OriginalTweet'] = train_df3['OriginalTweet'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')
/tmp/ipykernel_14697/2468492154.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  test_df3['OriginalTweet'] = test_df3['OriginalTweet'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')


In [20]:
train_df3.head()


,UserName,ScreenName,TweetAt,OriginalTweet,Sentiment
0,3799,48751,16-03-2020,and and,Neutral
1,3800,48752,16-03-2020,advice talk to your neighbours family to excha...,Positive
2,3801,48753,16-03-2020,coronavirus australia woolworths to give elder...,Positive
3,3802,48754,16-03-2020,my food stock is not the only one which is emp...,Positive
4,3803,48755,16-03-2020,me ready to go at supermarket during the outbr...,Extremely Negative


In [21]:
def give_number_to_3_class(row):
    sent = row['Sentiment']

    if sent=='Extremely Negative':
        return -1
    elif sent=='Negative':
        return -1
    elif sent=='Neutral':
        return 0
    elif sent=='Positive':
        return 1
    else:
        return 1

In [22]:
def give_number_to_class(row):
    sent = row['Sentiment']

    if sent=='Extremely Negative':
        return 0
    elif sent=='Negative':
        return 1
    elif sent=='Neutral':
        return 2
    elif sent=='Positive':
        return 3
    else:
        return 4


    

In [23]:
train_df3['Sentiment_Number'] = train_df3.apply(give_number_to_3_class, axis=1)
test_df3['Sentiment_Number'] = test_df3.apply(give_number_to_3_class, axis=1)



In [24]:
train_df3.head()

,UserName,ScreenName,TweetAt,OriginalTweet,Sentiment,Sentiment_Number
0,3799,48751,16-03-2020,and and,Neutral,0
1,3800,48752,16-03-2020,advice talk to your neighbours family to excha...,Positive,1
2,3801,48753,16-03-2020,coronavirus australia woolworths to give elder...,Positive,1
3,3802,48754,16-03-2020,my food stock is not the only one which is emp...,Positive,1
4,3803,48755,16-03-2020,me ready to go at supermarket during the outbr...,Extremely Negative,-1


In [25]:
np.unique(train_df3["Sentiment"].values)

array(['Extremely Negative', 'Extremely Positive', 'Negative', 'Neutral',
       'Positive'], dtype=object)

In [26]:
X_train = train_df3['OriginalTweet']  
X_test = test_df3['OriginalTweet']  

y_train = train_df3['Sentiment_Number']
y_test = test_df3['Sentiment_Number']

In [27]:
X_train[0]

'and and'

In [28]:
df_train.isnull().sum()

UserName         0
ScreenName       0
TweetAt          0
OriginalTweet    0
Sentiment        0
dtype: int64

In [29]:
df_train['TweetAt'].values

array(['16-03-2020', '16-03-2020', '16-03-2020', ..., '14-04-2020',
       '14-04-2020', '14-04-2020'], dtype=object)

# Recherche d'Hyperparamètres

In [30]:
y_train.shape

(41157,)

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [32]:
X_train


0                                                  and and
1        advice talk to your neighbours family to excha...
2        coronavirus australia woolworths to give elder...
3        my food stock is not the only one which is emp...
4        me ready to go at supermarket during the outbr...
                               ...                        
41152    airline pilots offering to stock supermarket s...
41153    response to complaint not provided citing covi...
41154    you know its getting tough when is rationing t...
41155    is it wrong that the smell of hand sanitizer i...
41156    well new used rift s are going for 700 00 on a...
Name: OriginalTweet, Length: 41157, dtype: object

### SGD

In [61]:


grid_params = {
  'clf__penalty': ['l2', 'l1', 'elasticnet'],
  'clf__alpha': np.linspace(0.00001, 0.001, 10),  
}



pipeline_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', SGDClassifier()),
])


sgd_clf = GridSearchCV(pipeline_clf, grid_params,verbose=1, scoring='accuracy' ,n_jobs=-1)








In [62]:
%%time
sgd_best = sgd_clf.fit(X_train, y_train)


Fitting 5 folds for each of 30 candidates, totalling 150 fits
CPU times: user 5.23 s, sys: 3.03 s, total: 8.26 s
Wall time: 22.8 s


In [63]:
print("Best Score: ", sgd_clf.best_score_)
print("Best Params: ", sgd_clf.best_params_)



# Feed the training data through the pipeline
#text_clf.fit(X_train, y_train) 


Best Score:  0.8765217187765468
Best Params:  {'clf__alpha': 1e-05, 'clf__penalty': 'l1'}


### Gradient boosting

In [46]:
from sklearn.ensemble import GradientBoostingClassifier

In [47]:
GB_param = {
    "gb_clf__learning_rate": [0.01, 0.1, 0.2],
    "gb_clf__n_estimators":[10,100]
    }

In [48]:
pipeline_gradient_boosting = Pipeline([('tfidf', TfidfVectorizer()),
                     ('gb_clf', GradientBoostingClassifier()),
])


gb_clf = GridSearchCV(pipeline_gradient_boosting, GB_param, scoring='accuracy', verbose=1 ,n_jobs=-1)

In [49]:
%%time
subset = X_train.shape[0]

gb_best = gb_clf.fit(X_train[:subset], y_train[:subset])    

Fitting 5 folds for each of 6 candidates, totalling 30 fits
CPU times: user 1min 13s, sys: 393 ms, total: 1min 13s
Wall time: 3min 36s


In [50]:
print("Best Score: ", gb_clf.best_score_)
print("Best Params: ", gb_clf.best_params_)

Best Score:  0.7134627865760047
Best Params:  {'gb_clf__learning_rate': 0.2, 'gb_clf__n_estimators': 100}


### Random Forest

In [51]:
randForest_param = {
    'rf_clf__n_estimators': [10, 100],   
    'rf_clf__max_depth': [10 ,300],
    'rf_clf__min_samples_leaf': [1, 2, 3],   
    'rf_clf__min_samples_split': [4, 8],
    'rf_clf__max_features': ['log2', 'sqrt'],
    'rf_clf__criterion': ['gini', 'entropy'],
    'rf_clf__warm_start': [True, False] 
}



pipeline_random_forest = Pipeline([('tfidf', TfidfVectorizer()),
                     ('rf_clf', RandomForestClassifier()),
])


rf_clf = GridSearchCV(pipeline_random_forest, randForest_param, scoring='accuracy', verbose=1 ,n_jobs=-1)


In [52]:
%%time
subset = X_train.shape[0]

rf_best = rf_clf.fit(X_train[:subset], y_train[:subset])



Fitting 5 folds for each of 192 candidates, totalling 960 fits
CPU times: user 35 s, sys: 16.5 s, total: 51.5 s
Wall time: 9min 50s


In [53]:
print("Best Score: ", rf_clf.best_score_)
print("Best Params: ", rf_clf.best_params_)

Best Score:  0.6947539369462834
Best Params:  {'rf_clf__criterion': 'gini', 'rf_clf__max_depth': 300, 'rf_clf__max_features': 'sqrt', 'rf_clf__min_samples_leaf': 2, 'rf_clf__min_samples_split': 8, 'rf_clf__n_estimators': 100, 'rf_clf__warm_start': False}


### Logistic Regression

In [54]:
LogisticRegression_param = {
    'lr_clf__C': [100, 10, 1.0, 0.1, 0.01],
    'lr_clf__tol': np.linspace(10**-6,10**-2,10)
}


In [55]:
pipeline_lr = Pipeline([('tfidf', TfidfVectorizer()),
                       
                     ('lr_clf', LogisticRegression(max_iter=1000)),
 ])

In [56]:
lr_clf = GridSearchCV(pipeline_lr, LogisticRegression_param,  scoring='accuracy', verbose=1, n_jobs=-1)

In [58]:
%%time
lr_best = lr_clf.fit(X_train[:subset], y_train[:subset])

Fitting 5 folds for each of 50 candidates, totalling 250 fits
CPU times: user 1min 34s, sys: 4min 16s, total: 5min 51s
Wall time: 10min 9s


In [59]:
subset

41157

In [60]:
print("Best Score: ", lr_clf.best_score_)
print("Best Params: ", lr_clf.best_params_)

Best Score:  0.8230920602963577
Best Params:  {'lr_clf__C': 10, 'lr_clf__tol': 1e-06}


### Perceptron

In [64]:
from sklearn.linear_model import Perceptron


In [65]:
per_params = {
  'per_clf__penalty': ['l2', 'l1', 'elasticnet'],
  'per_clf__alpha': np.linspace(0.00001, 0.001, 10),
}

In [66]:
pipeline_perceptron = Pipeline([('tfidf', TfidfVectorizer()),
                       
                     ('per_clf', Perceptron()),
 ])

In [67]:
per_clf = GridSearchCV(pipeline_perceptron, per_params,  scoring='accuracy', verbose=1, n_jobs=-1)

In [79]:
%%time
per_best = per_clf.fit(X_train[:subset], y_train[:subset])

Fitting 5 folds for each of 30 candidates, totalling 150 fits
CPU times: user 4.52 s, sys: 3.14 s, total: 7.65 s
Wall time: 19 s


In [80]:
print("Best Score: ", per_clf.best_score_)
print("Best Params: ", per_clf.best_params_)

Best Score:  0.791651326688233
Best Params:  {'per_clf__alpha': 1e-05, 'per_clf__penalty': 'l1'}


#### Observation 

La regression logistique n'est qu'un perceptron avec une sigmoid en fonction d'activation.
On voit que la Regression Logistique a de meilleures performances à l'issue de la recherche d'hyperparamètres mais pas de loin. Par ailleurs le temps d'entrainement est considérablement plus élevé pour la regression logistique (du au calcul de l'exponentiel). Nous verrons par la suite quel modèle il est préférable de conserver. 

### LinearSVC

In [70]:
from sklearn.svm import LinearSVC


In [71]:
svc_params = {
  'svc_clf__penalty': ['l2', 'l1', 'elasticnet'],
  'svc_clf__loss': ['hinge', 'squared_hinge'],
  'svc_clf__dual' : [False,True]
}

In [72]:
pipeline_svc = Pipeline([('tfidf', TfidfVectorizer()),
                       
                     ('svc_clf', LinearSVC(max_iter=10000)),
 ])

In [73]:
svc_clf = GridSearchCV(pipeline_svc, svc_params,  scoring='accuracy', verbose=1, n_jobs=-1)

In [78]:
%%time
svc_best = svc_clf.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/home/eliott/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
40 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/home/eliott/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/eliott/.local/lib/python3.10/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/home/eliott/.local/lib/python3.10/site-packages/sklearn/svm/_classes.py", line 257, in fit
    self.coef_, self.intercept_, self.n_it

CPU times: user 6.8 s, sys: 151 ms, total: 6.95 s
Wall time: 27 s


In [81]:
print("Best Score: ", svc_clf.best_score_)
print("Best Params: ", svc_clf.best_params_)

Best Score:  0.8689651633428769
Best Params:  {'svc_clf__dual': False, 'svc_clf__loss': 'squared_hinge', 'svc_clf__penalty': 'l1'}


In [82]:
import json

In [83]:
dict = {}
dict['SGD'] = sgd_clf.best_params_
dict['RF'] = rf_clf.best_params_
dict['GB'] = gb_clf.best_params_
dict['LR'] = lr_clf.best_params_
dict['Perceptron'] = per_clf.best_params_
dict['SVC'] = svc_clf.best_params_


with open('data.json', 'w') as fp:
    json.dump(dict, fp,  indent=4)

### Charging the best parameters for training

In [84]:
# Opening JSON file
with open('data.json') as json_file:
    data = json.load(json_file)


data_sgd = data['SGD']
data_rf = data['RF']
data_gb = data['GB']
data_lr = data['LR']
data_per = data['Perceptron']
data_svc = data['SVC']



### Training on all training data and testing on test data

In [85]:
from sklearn.metrics import accuracy_score, f1_score

In [99]:
model_sgd = pipeline_clf
model_sgd.set_params(**data_sgd)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', SGDClassifier(alpha=1e-05, penalty='l1'))])

In [100]:
%%time

model_sgd.fit(X_train,y_train,epochs=1000)


CPU times: user 1.91 s, sys: 2.69 s, total: 4.6 s
Wall time: 1.42 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', SGDClassifier(alpha=1e-05, penalty='l1'))])

In [102]:
predictions_train_sgd = model_sgd.predict(X_train)
predictions_test_sgd = model_sgd.predict(X_test)
accuracy_train = accuracy_score(y_train,predictions_train_sgd )
accuracy_test = accuracy_score(y_test,predictions_test_sgd )


print(accuracy_train)
print(accuracy_test)


0.9439949461816944
0.8812532912058978


In [ ]:
f1score = f1_score(y_test, ad_best_ss.predict(X_test_ss), average='weighted')


In [ ]:
predictions = text_clf.predict(X_test)

NameError: name 'text_clf' is not defined

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

train_acc = round(accuracy_score(predictions, y_test)*100)
train_acc

In [ ]:
len(y_test)

In [ ]:
print(classification_report(predictions, y_test, target_names=train_df3['Sentiment'].unique()))

In [ ]:
X_test[0]

'trending new yorkers encounter empty supermarket shelves pictured wegmans in brooklyn sold out online grocers foodkick maxdelivery as coronavirus fearing shoppers stock up https t co gr76pcrlwh https t co ivmkmsqdt1'

In [ ]:
# ACCURACY
c=0
num=len(predictions)
for k in range(num):
    if(predictions[k]==y_test[k]):
        c+=1

print(f"pourcentage de reussite : {(c*100)/num} ")




NameError: ignored

In [ ]:
# L2 ERROR

num=len(predictions)
err_tot = 0
for k in range(num):
    #print(predictions[k])
    if(k<20):
        print(f"prediction {predictions[k]}  , realité : {y_test[k]}")
    err_tot += np.square(predictions[k]-y_test[k])
err_tot/=num
err_tot = np.sqrt(err_tot) 
err_tot